In [24]:
from opt_einsum import contract
import numpy as np
import torch
from dataclasses import dataclass,asdict
from math import prod
from torch.linalg import svd as mysvd
from torch.linalg import eig as myeig
from tqdm.auto import tqdm
from utils import show_tensor_ijkl
import matplotlib.pyplot as plt
from copy import deepcopy
import itertools as itt
device=torch.device('cuda:0')
torch.set_default_tensor_type(torch.cuda.DoubleTensor)
torch.cuda.set_device(device)

In [16]:
def get_Ising1D_Hamiltonians(J,g):
    sZ=torch.tensor([[1,0],[0,-1]])
    sX=torch.tensor([[0,1],[1,0]])
    eye=torch.eye(2)
    ZZ=contract('iI,jJ->ijIJ',sZ,sZ)
    XI=contract('iI,jJ->ijIJ',sX,eye)
    IX=contract('iI,jJ->ijIJ',eye,sX)
    h=-J*(ZZ+g*(XI+IX)/2)
    return h,h.clone()
def get_Ising1D_Hamiltonians_2blocked(J,g):
    sZ=torch.tensor([[1,0],[0,-1]])
    sX=torch.tensor([[0,1],[1,0]])
    eye=torch.eye(2)
    ZZII=contract('iI,jJ,kK,lL->ijklIJKL',sZ,sZ,eye,eye)
    IZZI=contract('iI,jJ,kK,lL->ijklIJKL',eye,sZ,sZ,eye)
    IIZZ=contract('iI,jJ,kK,lL->ijklIJKL',eye,eye,sZ,sZ)
    XIII=contract('iI,jJ,kK,lL->ijklIJKL',sX,eye,eye,eye)
    IXII=contract('iI,jJ,kK,lL->ijklIJKL',eye,sX,eye,eye)
    IIXI=contract('iI,jJ,kK,lL->ijklIJKL',eye,eye,sX,eye)
    IIIX=contract('iI,jJ,kK,lL->ijklIJKL',eye,eye,eye,sX)
    h=((-J*((ZZII+IIZZ)/2+IZZI)-J*g*(XIII+IXII+IIXI+IIIX)/2)/2).reshape(2,2,2,2,2,2,2,2)
    # need to take care of the reflection symmetry!
    hAB=contract('abcdABCD->abdcABDC',h).reshape(4,4,4,4)
    hBA=contract('abcdABCD->bacdBACD',h).reshape(4,4,4,4)
    return hAB, hBA



In [20]:
# reference https://www.tensors.net/mera

@dataclass
class MERAOptions:
    reflection_symmetry:bool=True
    nLayers:int=2
    max_dim_mid:int=4
    max_dim:int=6
    nSweeps:int=100
    nIterRhoTop:int=4

@dataclass
class MERALayer:
    '''
    1 2 1     1 1 2 1 2
    uuu www vvv rho hhh
    3 4 2 3 2 3 3 4 3 4

    w     vvv     w
    uuuuuuu |     |
    B     A B     A
    www vvv www vvv
    | uuu | | uuu |
    A B A B A B A B
    
    [rhoBA]  [rhoBA]  [rhoBA]  [rhoAB]
    www vvv  www vvv  www vvv  vvv www
    | uuu |  | uuu |  | uuu |  | | | |
    hAB | |  | hBA |  | | hAB  | hBA |
    [conj ]  [conj ]  [conj ]  [conj ]
    '''
    u: torch.Tensor
    w: torch.Tensor
    v: torch.Tensor
    rhoAB: torch.Tensor=torch.tensor(0.)
    rhoBA: torch.Tensor=torch.tensor(0.)
    hAB: torch.Tensor=torch.tensor(0.)
    hBA: torch.Tensor=torch.tensor(0.)
    h_bias: torch.Tensor=torch.tensor(0.)
    @property
    def d(self)->int:# dimension of the fine-grained space
        return self.w.shape[1]
    @property
    def D_mid(self)->int:# dimension of the fine-grained space after u
        return self.w.shape[2]
    @property
    def D(self)->int:# dimension of the coarse-grained space
        return self.w.shape[0]
    def unpack(layer,dtype=None):
        u,w,v=layer.u,layer.w,layer.v
        rhoAB,rhoBA=layer.rhoAB,layer.rhoBA
        hAB,hBA=layer.hAB,layer.hBA
        uc,wc,vc=torch.conj(u).clone(),torch.conj(w).clone(),torch.conj(v).clone()
        eye=torch.eye(layer.d,device=device)
        eyem=torch.eye(layer.D_mid,device=device)
        rtval=u,w,v,rhoAB,rhoBA,hAB,hBA,uc,wc,vc,eye,eyem
        if dtype is not None:
            rtval=tuple(t.to(dtype) for t in rtval)
        return rtval

def dcontract(derivative,eq,*tensors,**kwargs):
    assert all(tensor is not None for tensor in tensors)
    assert len(list(tensor for tensor in tensors if id(tensor)==id(derivative)))==1, f'{id(derivative)%3533} {[id(t)%3533 for t in tensors]}'
    idx = next(i for i, tensor in enumerate(tensors) if id(tensor)==id(derivative))
    eq_terms=eq.split(',')
    eq=','.join(eq_terms[:idx]+eq_terms[idx+1:])+'->'+eq_terms[idx]
    tensors=tensors[:idx]+tensors[idx+1:]
    return contract(eq,*tensors,**kwargs)

def get_energy(layer:MERALayer,options:MERAOptions):
    u,w,v,rhoAB,rhoBA,hAB,hBA,uc,wc,vc,eye,eyem=layer.unpack()
    term1=contract('IJij,ikl,jmn,lmop,IKL,JMN,LMOP,koKO,pP,nN',
                    rhoBA,w,v,u,wc,vc,uc,hAB,eye,eye)
    term2=contract('IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,opOP,nN',
                    rhoBA,w,v,u,wc,vc,uc,eye,hBA,eye)
    term3=contract('IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,oO,pnPN',
                    rhoBA,w,v,u,wc,vc,uc,eye,eye,hAB) \
            if not options.reflection_symmetry else term1
    term4=contract('IJij,ikl,jmn,IKL,JMN,kK,lmLM,nN',
                    rhoAB,v,w,vc,wc,eyem,hBA,eyem)
    return (term1+term2+term3+term4)/4+layer.h_bias


def propogate_down(layer:MERALayer,options:MERAOptions):
    u,w,v,rhoAB,rhoBA,hAB,hBA,uc,wc,vc,eye,eyem=layer.unpack()
    # print('propogate_down')
    # print(rhoBA.shape,w.shape,v.shape,u.shape,hAB.shape)
    rhoAB1=dcontract(hAB,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,koKO,pP,nN',
                    rhoBA,w,v,u,wc,vc,uc,hAB,eye,eye)
    rhoBA1=dcontract(hBA,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,opOP,nN',
                    rhoBA,w,v,u,wc,vc,uc,eye,hBA,eye)
    rhoAB2=dcontract(hAB,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,oO,pnPN',
                    rhoBA,w,v,u,wc,vc,uc,eye,eye,hAB)\
            if not options.reflection_symmetry else contract('ijIJ->jiJI',rhoAB1)
    rhoBA2=dcontract(hBA,'IJij,ikl,jmn,IKL,JMN,kK,lmLM,nN',
                    rhoAB,v,w,vc,wc,eyem,hBA,eyem)
    new_rhoAB=rhoAB1+rhoAB2
    new_rhoBA=rhoBA1+rhoBA2
    new_rhoAB=new_rhoAB/contract('ijij',new_rhoAB)
    new_rhoBA=new_rhoBA/contract('ijij',new_rhoBA)
    return new_rhoAB,new_rhoBA

def propogate_up(layer:MERALayer,options:MERAOptions):
    u,w,v,rhoAB,rhoBA,hAB,hBA,uc,wc,vc,eye,eyem=layer.unpack()
    # print('propogate_up')
    # print(rhoBA.shape,v.shape,w.shape,vc.shape,wc.shape,hAB.shape)
    hBA1=dcontract(rhoBA,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,koKO,pP,nN',
                    rhoBA,w,v,u,wc,vc,uc,hAB,eye,eye)
    hBA2=dcontract(rhoBA,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,opOP,nN',
                    rhoBA,w,v,u,wc,vc,uc,eye,hBA,eye)
    hBA3=dcontract(rhoBA,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,oO,pnPN',
                    rhoBA,w,v,u,wc,vc,uc,eye,eye,hAB)\
            if not options.reflection_symmetry else contract('ijIJ->jiJI',hBA1)
    hAB1=dcontract(rhoAB,'IJij,ikl,jmn,IKL,JMN,kK,lmLM,nN',
                    rhoAB,v,w,vc,wc,eyem,hBA,eyem)
    new_hAB=hAB1/2
    new_hBA=(hBA1+hBA2+hBA3)/2
    return new_hAB,new_hBA


def get_u_env(layer:MERALayer,options:MERAOptions):
    u,w,v,rhoAB,rhoBA,hAB,hBA,uc,wc,vc,eye,eyem=layer.unpack()
    term1=dcontract(u,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,koKO,pP,nN',
                    rhoBA,w,v,u,wc,vc,uc,hAB,eye,eye)
    term2=dcontract(u,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,opOP,nN',
                    rhoBA,w,v,u,wc,vc,uc,eye,hBA,eye)
    term3=dcontract(u,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,oO,pnPN',
                    rhoBA,w,v,u,wc,vc,uc,eye,eye,hAB)\
            if not options.reflection_symmetry else contract('ijIJ->jiJI',term1)
    return (term1+term2+term3)/4

def get_w_env(layer:MERALayer,options:MERAOptions):
    u,w,v,rhoAB,rhoBA,hAB,hBA,uc,wc,vc,eye,eyem=layer.unpack()
    term1=dcontract(w,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,koKO,pP,nN',
                    rhoBA,w,v,u,wc,vc,uc,hAB,eye,eye)
    term2=dcontract(w,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,opOP,nN',
                    rhoBA,w,v,u,wc,vc,uc,eye,hBA,eye)
    term3=dcontract(w,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,oO,pnPN',
                    rhoBA,w,v,u,wc,vc,uc,eye,eye,hAB)
    term4=dcontract(w,'IJij,ikl,jmn,IKL,JMN,kK,lmLM,nN',
                    rhoAB,v,w,vc,wc,eyem,hBA,eyem)
    return (term1+term2+term3+term4)/4

def get_v_env(layer:MERALayer,options:MERAOptions):
    u,w,v,rhoAB,rhoBA,hAB,hBA,uc,wc,vc,eye,eyem=layer.unpack()
    term1=dcontract(v,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,koKO,pP,nN',
                    rhoBA,w,v,u,wc,vc,uc,hAB,eye,eye)
    term2=dcontract(v,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,opOP,nN',
                    rhoBA,w,v,u,wc,vc,uc,eye,hBA,eye)
    term3=dcontract(v,'IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,oO,pnPN',
                    rhoBA,w,v,u,wc,vc,uc,eye,eye,hAB)
    term4=dcontract(v,'IJij,ikl,jmn,IKL,JMN,kK,lmLM,nN',
                    rhoAB,v,w,vc,wc,eyem,hBA,eyem)
    return (term1+term2+term3+term4)/4



def svd_tensor_to_isometry(M,idx1,idx2):
    shape1=tuple(M.shape[i] for i in idx1)
    shape2=tuple(M.shape[i] for i in idx2)
    M=M.permute(idx1+idx2).reshape(prod(shape1),prod(shape2))
    u,_,vh=mysvd(M,full_matrices=False)
    uvh=(u@vh).reshape(shape1+shape2).permute(tuple(np.argsort(idx1+idx2)))
    return uvh

def get_isometry_from_environment(M,idx1,idx2):
    return svd_tensor_to_isometry(M,idx1,idx2).conj()

def optimize_layer_(layer:MERALayer,options:MERAOptions):
    uenv=get_u_env(layer,options)
    wenv=get_w_env(layer,options)
    if options.reflection_symmetry:
        layer.u=get_isometry_from_environment(uenv+contract('ijIJ->jiJI',uenv),[0,1],[2,3])
        layer.w=get_isometry_from_environment(wenv,[0],[1,2])
        layer.v=contract('ijk->ikj',layer.w.clone())
    else:
        venv=get_v_env(layer,options)
        layer.u=get_isometry_from_environment(uenv,[0,1],[2,3])
        layer.w=get_isometry_from_environment(wenv,[0],[1,2])
        layer.v=get_isometry_from_environment(venv,[0],[1,2])


def init_layers(hAB,hBA,options:MERAOptions):
    d=hAB.shape[0]
    layers=[]
    # make sure hAB and hBA are negative semidefinite
    h_bias=torch.maximum(
        torch.linalg.eigvalsh(hAB.reshape((d**2,d**2))).max(),
        torch.linalg.eigvalsh(hBA.reshape((d**2,d**2))).max()
    )
    hAB=hAB-h_bias*torch.eye(d**2).reshape((d,d,d,d))
    hBA=hBA-h_bias*torch.eye(d**2).reshape((d,d,d,d))

    # generate the layers
    for i in range(options.nLayers):
        Dm=min(d,options.max_dim_mid)
        D=min(d**2,options.max_dim)
        # here we don't need to generate unitary
        u=torch.eye(Dm**2,d**2).reshape((Dm,Dm,d,d))
        w=torch.randn((D,d,Dm))
        v=torch.randn((D,Dm,d))
        layers.append(MERALayer(u=u,w=w,v=v,h_bias=h_bias))
        d=D

    layers[0].hAB=hAB.clone()
    layers[0].hBA=hBA.clone()
    layers[-1].rhoAB=torch.eye(D**2).reshape((D,D,D,D))
    layers[-1].rhoBA=torch.eye(D**2).reshape((D,D,D,D))

    return layers

def pad_tensor(A:torch.tensor,shape):
    '''expand tensor dimension by padding with zeros'''
    for k in range(len(shape)):
        if A.shape[k] != shape[k]:
            A=torch.cat([A,torch.zeros(A.shape[:k]+(shape[k]-A.shape[k],)+A.shape[k+1:])],dim=k)
    return A

def pad_layers_(layers:'list[MERALayer]',options:MERAOptions):
    d=layers[0].d
    for i in range(options.nLayers):
        Dm=min(d,options.max_dim_mid)
        D=min(d**2,options.max_dim)
        if i>=len(layers):
            # generate the layers
            layers.append(MERALayer(
                u=layers[-1].u.clone(),
                w=layers[-1].w.clone(),
                v=layers[-1].v.clone(),
                h_bias=layers[-1].h_bias,
                rhoAB=layers[-1].rhoAB.clone(),
                rhoBA=layers[-1].rhoBA.clone(),
                hAB=layers[-1].hAB.clone(),
                hBA=layers[-1].hBA.clone()
            ))
                                    
        # pad the tensors
        layers[i].u=pad_tensor(layers[i].u,(Dm,Dm,d,d))
        layers[i].w=pad_tensor(layers[i].w,(D,d,Dm))
        layers[i].v=pad_tensor(layers[i].v,(D,Dm,d))
        layers[i].rhoAB=pad_tensor(layers[i].rhoAB,(D,D,D,D))
        layers[i].rhoBA=pad_tensor(layers[i].rhoBA,(D,D,D,D))
        layers[i].hAB=pad_tensor(layers[i].hAB,(d,d,d,d))
        layers[i].hBA=pad_tensor(layers[i].hBA,(d,d,d,d))
        d=D
    # reset the density operator at the top layer
    layers[-1].rhoAB=torch.eye(D**2).reshape((D,D,D,D))
    layers[-1].rhoBA=torch.eye(D**2).reshape((D,D,D,D))

def sweep_layers(layers:'list[MERALayer]',options:MERAOptions):
    # determine the scale-invariant density operator
    layer=layers[-1]
    assert layer.D==layer.d
    # use the previous density operator as the initial guess
    for i in range(options.nIterRhoTop):
        layer.rhoAB,layer.rhoBA=propogate_down(layer,options)
    # propogate the density operators down
    for i in range(options.nLayers-1,-1,-1):
        # need to sweep back and forth to ensure convergence
        # in GE's code, he only optimize at the up sweep
        # maybe it will help jumping out local minimum?
        # if len(layers[i].hAB.shape)>0:
        #     optimize_layer_(layers[i],options)
        if i>0:
            layers[i-1].rhoAB,layers[i-1].rhoBA=propogate_down(layers[i],options)
    # propogate the Hamiltonians up
    for i in range(options.nLayers):
        optimize_layer_(layers[i],options)
        if i<options.nLayers-1:
            layers[i+1].hAB,layers[i+1].hBA=propogate_up(layers[i],options)



In [80]:
def renormalize_op(layer,op,scdim):
    '''
    we set renormalized lattice constance = 0.5
    [rhoBA    ]
    [www] [vvv]
    |   [u]   |
    op Id op Id = C(op,op)/(1)^(2 Δ)
    [conj     ]
    '''
    u,w,v,rhoAB,rhoBA,hAB,hBA,uc,wc,vc,eye,eyem=layer.unpack(op.dtype)
    C00=contract('IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,oO,pP,nN->',
               rhoBA,w,v,u,wc,vc,uc,op,eye,op,eye)
    return op*C00.abs()**-.5


def get_conformal_ops(layer,k=10):
    '''
      |
    vvv
    | www
    | O |
    [conj]
    '''
    assert layer.d==layer.D
    w,v,wc,vc=layer.w,layer.v,torch.conj(layer.w),torch.conj(layer.v)
    d=layer.d
    M=contract('ijk,klm,IjK,KLm->iIlL',v,w,vc,wc)
    s,u=torch.linalg.eig(M.reshape((d**2,d**2)))
    s,u=s[torch.argsort(s.abs(),descending=True)],u[:,torch.argsort(s.abs(),descending=True)]
    scdims=torch.log2(s[:k].abs()).abs()/2
    ops=[u[:,i].reshape((d,d)) for i in range(k)]
    ops=[renormalize_op(layer,op,scdim) for op,scdim in zip(ops,scdims)]
    return ops,scdims



def get_ope_coeff(layer,ops,scdims):
    '''
    [vvv] [www] [vvv] [www]
    Id Id o0 Id o1 Id o2 id = C(o0,o1,o2) / 2^(Δ0+Δ2-Δ1)
    '''
    u,w,v,rhoAB,rhoBA,hAB,hBA,uc,wc,vc,eye,eyem=layer.unpack(ops[0].dtype)
    op01_level1=contract('ikl,jmn,lmop,IKL,JMN,LMOP,kK,oO,pP,nN->ijIJ',
                    w,v,u,wc,vc,uc,ops[0],eye,ops[1],eye)
    op2_level1=contract('ijk,IJK,jJ,kK->iI',
                    w,wc,ops[2],eyem) # we also need to propogate op2 to the next layer
    expval=contract('IJij,ikl,jmn,lmop,IKL,JMN,LMOP,kK,opOP,nN->',
                    rhoBA,w,v,u,wc,vc,uc,eye,op01_level1,op2_level1)
    C012=expval*2**(scdims[0]-scdims[1]+scdims[2])
    return C012
    
    

def troubleshoot(layers:'list[MERALayer]',options:MERAOptions,verbose=2):
    global energy_ref, scdim_ref
    energy=get_energy(layers[0],options).detach().cpu().item()
    energyErr=abs(energy-energy_ref)
    print('Energy', '%.7f'%energy, 'error', '%.2e'%energyErr)
    if verbose>0:
        ops,scdims=get_conformal_ops(layers[-1],k=10)
        print('scdims', scdims.detach().cpu().numpy().round(3))
        if verbose>1:
            print('ref   ', np.array(scdim_ref)[:len(scdims)].round(3))

        opes={}
        for i,j,k in itt.product(range(3),repeat=3):
            opes[(i,j,k)]=get_ope_coeff(layers[-1],[ops[i],ops[j],ops[k]],[scdims[i],scdims[j],scdims[k]])

        if verbose>1:
            print('ope coeffs')
            for i,j,k in itt.combinations_with_replacement(range(3),3):
                for i1,j1,k1 in sorted(set(itt.permutations([i,j,k]))):
                    print('C_'+str(i1)+str(j1)+str(k1), '%.3f'%opes[(i1,j1,k1)].abs(), end='\t')
                print()
        else:
            print('C_112', '%.3f'%opes[(1,1,2)].abs())

In [81]:

J,g=1,1
energy_ref=-4/np.pi
scdim_ref=[0,.125,1]+[1.125]*2+[2]*4+[2.125]*3+[3]*5+[3.125]*6+[4]*9+[4.125]*9+[5]*13+[5.125]*14
hAB,hBA=get_Ising1D_Hamiltonians_2blocked(J,g)
print('J=%f, g=%f'%(J,g))
print('Exact energy=%f'%energy_ref)

filename='./data/Ising1DMERA.pth'
optionss=[
    MERAOptions(nLayers=2,max_dim=6,max_dim_mid=4,nIterRhoTop=4,nSweeps=2000),
    MERAOptions(nLayers=3,max_dim=8,max_dim_mid=6,nIterRhoTop=4,nSweeps=1000),
    MERAOptions(nLayers=4,max_dim=12,max_dim_mid=8,nIterRhoTop=4,nSweeps=1000),
    MERAOptions(nLayers=5,max_dim=16,max_dim_mid=12,nIterRhoTop=4,nSweeps=500),
    MERAOptions(nLayers=6,max_dim=20,max_dim_mid=16,nIterRhoTop=4,nSweeps=100),
]
for ii,options in enumerate(optionss):
    print('sweep using the following options: ')
    print(asdict(options))
    if ii==0:
        layers=init_layers(hAB,hBA,options)
    else:
        pad_layers_(layers,options)
    for i in tqdm(range(1,1+options.nSweeps)):
        sweep_layers(layers,options)
        if i%(options.nSweeps//10)==0:
            print('Sweep',i)
            troubleshoot(layers,options,verbose=1)
            torch.save((layers,options), filename)
            print('saved to', filename)
    troubleshoot(layers,options,verbose=2)
    torch.save((layers,options), filename)
    print('saved to', filename)

J=1.000000, g=1.000000
Exact energy=-1.273240
sweep using the following options: 
{'reflection_symmetry': True, 'nLayers': 2, 'max_dim_mid': 4, 'max_dim': 6, 'nSweeps': 2000, 'nIterRhoTop': 4}


  0%|          | 0/2000 [00:00<?, ?it/s]

Sweep 200
Energy -1.2731111 error 1.28e-04
scdims [0.    0.134 1.123 1.21  1.268 1.742 2.057 2.061 2.188 2.218]
C_112 0.590
saved to ./data/Ising1DMERA.pth
Sweep 400
Energy -1.2732051 error 3.44e-05
scdims [0.    0.13  1.07  1.131 1.217 1.885 1.965 1.987 2.039 2.119]
C_112 0.525
saved to ./data/Ising1DMERA.pth
Sweep 600
Energy -1.2732131 error 2.64e-05
scdims [0.    0.128 1.061 1.132 1.186 1.911 1.917 1.968 2.091 2.14 ]
C_112 0.513
saved to ./data/Ising1DMERA.pth
Sweep 800
Energy -1.2732153 error 2.42e-05
scdims [0.    0.127 1.06  1.138 1.176 1.903 1.918 1.97  2.085 2.241]
C_112 0.510
saved to ./data/Ising1DMERA.pth
Sweep 1000
Energy -1.2732164 error 2.32e-05
scdims [0.    0.127 1.061 1.143 1.172 1.902 1.919 1.975 2.082 2.311]
C_112 0.510
saved to ./data/Ising1DMERA.pth
Sweep 1200
Energy -1.2732170 error 2.25e-05
scdims [0.    0.127 1.061 1.146 1.169 1.903 1.92  1.978 2.078 2.359]
C_112 0.510
saved to ./data/Ising1DMERA.pth
Sweep 1400
Energy -1.2732175 error 2.20e-05
scdims [0.    0.12

  0%|          | 0/1000 [00:00<?, ?it/s]

Sweep 100
Energy -1.2732376 error 1.99e-06
scdims [0.    0.125 1.005 1.09  1.152 1.795 1.924 1.935 1.936 1.968]
C_112 0.500
saved to ./data/Ising1DMERA.pth
Sweep 200
Energy -1.2732378 error 1.75e-06
scdims [0.    0.125 1.    1.083 1.15  1.783 1.913 1.927 1.929 1.952]
C_112 0.499
saved to ./data/Ising1DMERA.pth
Sweep 300
Energy -1.2732379 error 1.63e-06
scdims [0.    0.125 0.999 1.082 1.148 1.782 1.912 1.927 1.928 1.949]
C_112 0.499
saved to ./data/Ising1DMERA.pth
Sweep 400
Energy -1.2732380 error 1.54e-06
scdims [0.    0.124 0.998 1.082 1.148 1.783 1.914 1.928 1.929 1.947]
C_112 0.499
saved to ./data/Ising1DMERA.pth
Sweep 500
Energy -1.2732381 error 1.46e-06
scdims [0.    0.124 0.998 1.082 1.147 1.785 1.916 1.929 1.931 1.946]
C_112 0.499
saved to ./data/Ising1DMERA.pth
Sweep 600
Energy -1.2732382 error 1.39e-06
scdims [0.    0.125 0.998 1.083 1.147 1.787 1.918 1.93  1.932 1.946]
C_112 0.499
saved to ./data/Ising1DMERA.pth
Sweep 700
Energy -1.2732382 error 1.33e-06
scdims [0.    0.125 0

...

  0%|          | 0/500 [00:00<?, ?it/s]

Sweep 50
Energy -1.2732395 error 4.47e-08
scdims [0.    0.125 0.998 1.117 1.121 1.933 1.948 1.983 1.984 1.992]
C_112 0.501
saved to ./data/Ising1DMERA.pth
Sweep 100
Energy -1.2732395 error 3.70e-08
scdims [0.    0.125 0.998 1.118 1.121 1.942 1.954 1.979 1.987 1.989]
C_112 0.501
saved to ./data/Ising1DMERA.pth
Sweep 150
Energy -1.2732395 error 3.34e-08
scdims [0.    0.125 0.999 1.118 1.121 1.936 1.95  1.971 1.991 2.   ]
C_112 0.501
saved to ./data/Ising1DMERA.pth
Sweep 200
Energy -1.2732395 error 3.10e-08
scdims [0.    0.125 0.999 1.119 1.121 1.907 1.957 1.96  1.994 2.014]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 250
Energy -1.2732395 error 2.93e-08
scdims [0.    0.125 0.999 1.12  1.121 1.881 1.951 1.963 1.997 2.018]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 300
Energy -1.2732395 error 2.80e-08
scdims [0.    0.125 0.999 1.121 1.121 1.865 1.945 1.969 2.    2.016]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 350
Energy -1.2732395 error 2.70e-08
scdims [0.    0.125 0.

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_1417863/604513450.py:162: UserWarning: torch.linalg.svd: During SVD computation with the selected cusolver driver, batches 0 failed to converge. A more accurate method will be used to compute the SVD as a fallback. Check doc at https://pytorch.org/docs/stable/generated/torch.linalg.svd.html (Triggered internally at ../aten/src/ATen/native/cuda/linalg/BatchLinearAlgebraLib.cpp:869.)
  u,_,vh=mysvd(M,full_matrices=False)


Sweep 10
Energy -1.2732395 error 1.97e-08
scdims [0.    0.125 1.    1.12  1.123 1.858 1.953 1.985 2.007 2.014]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 20
Energy -1.2732395 error 1.67e-08
scdims [0.    0.125 1.    1.12  1.123 1.862 1.955 1.985 2.007 2.015]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 30
Energy -1.2732395 error 1.53e-08
scdims [0.    0.125 1.    1.12  1.123 1.866 1.957 1.986 2.006 2.016]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 40
Energy -1.2732395 error 1.44e-08
scdims [0.    0.125 1.    1.12  1.123 1.869 1.958 1.986 2.006 2.017]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 50
Energy -1.2732395 error 1.38e-08
scdims [0.    0.125 1.    1.12  1.123 1.872 1.959 1.987 2.006 2.018]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 60
Energy -1.2732395 error 1.34e-08
scdims [0.    0.125 1.    1.121 1.123 1.874 1.96  1.987 2.006 2.018]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 70
Energy -1.2732395 error 1.31e-08
scdims [0.    0.125 1.    1.

# Compare with GE's code

In [49]:
def get_GE_Hamiltonians():
    sX = np.array([[0, 1], [1, 0]], dtype=float)
    sZ = np.array([[1, 0], [0, -1]], dtype=float)
    htemp = -np.kron(sX, sX) - 0.5 * (
        np.kron(sZ, np.eye(2)) + np.kron(np.eye(2), sZ))
    hbig = (0.5 * np.kron(np.eye(4), htemp) +
            np.kron(np.eye(2), np.kron(htemp, np.eye(2))) +
            0.5 * np.kron(htemp, np.eye(4))).reshape(2, 2, 2, 2, 2, 2, 2, 2)
    hamAB = (hbig.transpose(0, 1, 3, 2, 4, 5, 7, 6)).reshape(4, 4, 4, 4)
    hamBA = (hbig.transpose(1, 0, 2, 3, 5, 4, 6, 7)).reshape(4, 4, 4, 4)
    hamAB=torch.tensor(hamAB)/2
    hamBA=torch.tensor(hamBA)/2
    return hamAB,hamBA

def load_GE_data(path):
    ws, us, vs, rhoABs, rhoBAs=np.load(path,allow_pickle=True)
    print('dtype',ws[0].dtype)
    options=MERAOptions(
        nLayers=len(ws),
        max_dim=ws[-1].shape[2],
        max_dim_mid=ws[-1].shape[1])
    layers=[]
    hAB,hBA=get_GE_Hamiltonians()
    d=4
    h_bias=torch.maximum(
        torch.linalg.eigvalsh(hAB.reshape((d**2,d**2))).max(),
        torch.linalg.eigvalsh(hBA.reshape((d**2,d**2))).max()
    )
    hAB,hBA=hAB-h_bias*torch.eye(d**2).reshape((d,d,d,d)),hBA-h_bias*torch.eye(d**2).reshape((d,d,d,d))
    for i in range(len(ws)):
        w,v,u,rhoAB,rhoBA=ws[i],vs[i],us[i],rhoABs[i+1],rhoBAs[i+1]
        w,v,u,rhoAB,rhoBA=torch.tensor(w),torch.tensor(v),torch.tensor(u),torch.tensor(rhoAB),torch.tensor(rhoBA)
        w=contract('123->312',w)
        v=contract('123->321',v)
        u=contract('1234->3412',u)
        rhoAB=contract('1234->1234',rhoAB)
        rhoBA=contract('1234->1234',rhoBA)
        layers.append(MERALayer(w=w,u=u,v=v,rhoAB=rhoAB,rhoBA=rhoBA,hAB=hAB,hBA=hBA,h_bias=h_bias))
        hAB,hBA=propogate_up(layers[-1],options)
    return layers,options

layers_GE,options_GE=load_GE_data('./IsingData.npy')


dtype float64


In [336]:
layers,options=deepcopy(layers_GE),deepcopy(options_GE)


rhoAB0,rhoBA0=layers[-1].rhoAB.clone(),layers[-1].rhoBA.clone()
for i in range(10):
    layers[-1].rhoAB,layers[-1].rhoBA=propogate_down(layers[-1],options)
    rhoAB,rhoBA=layers[-1].rhoAB,layers[-1].rhoBA
    print('%.2e'%torch.norm(rhoAB-rhoAB0).item(),'%.2e'%torch.norm(rhoBA-rhoBA0).item())



1.09e-06 9.47e-07
9.92e-07 1.08e-06
1.12e-06 1.16e-06
1.03e-06 1.09e-06
1.11e-06 1.15e-06
1.04e-06 1.09e-06
1.11e-06 1.15e-06
1.04e-06 1.09e-06
1.10e-06 1.14e-06
1.04e-06 1.09e-06


In [337]:

layers=deepcopy(layers_GE)
options=deepcopy(options_GE)

troubleshoot(layers,options)
options.nSweeps=100
for i in tqdm(range(1,1+options.nSweeps)):
    sweep_layers(layers,options)
    if i%20==0 or i<=4:
        print('Sweep',i)
        troubleshoot(layers,options)

Energy -1.2732394 error 1.55e-07
scdim [-0.     0.125  1.005  1.12   1.13   1.763  1.856  1.949  1.965  2.082]
energies [-1.2732394 -1.2732394 -1.2732394 -1.2732394]


  0%|          | 0/100 [00:00<?, ?it/s]

Sweep 1
Energy -1.2732394 error 1.55e-07
scdim [-0.     0.125  1.005  1.12   1.13   1.763  1.856  1.949  1.965  2.082]
energies [-1.2732394 -1.2732394 -1.2732394 -1.2732394]
Sweep 2
Energy -1.2732394 error 1.55e-07
scdim [0.    0.125 1.005 1.12  1.13  1.763 1.856 1.949 1.965 2.082]
energies [-1.2732394 -1.2732394 -1.2732394 -1.2732394]
Sweep 3
Energy -1.2732394 error 1.55e-07
scdim [-0.     0.125  1.005  1.12   1.13   1.763  1.856  1.949  1.965  2.082]
energies [-1.2732394 -1.2732394 -1.2732394 -1.2732394]
Sweep 4
Energy -1.2732394 error 1.55e-07
scdim [-0.     0.125  1.005  1.12   1.13   1.763  1.856  1.949  1.965  2.082]
energies [-1.2732394 -1.2732394 -1.2732394 -1.2732394]
Sweep 20
Energy -1.2732394 error 1.54e-07
scdim [-0.     0.125  1.005  1.12   1.13   1.765  1.858  1.95   1.966  2.082]
energies [-1.2732394 -1.2732394 -1.2732394 -1.2732394]
Sweep 40
Energy -1.2732394 error 1.54e-07
scdim [0.    0.125 1.005 1.12  1.13  1.768 1.86  1.95  1.967 2.082]
energies [-1.2732394 -1.27323

KeyboardInterrupt: 